## Category: 推薦系統
### http://ihong-blog.logdown.com/categories/%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1


## [吳恩達的推薦系統課程筆記](http://www.holehouse.org/mlclass/16_Recommender_Systems.html)

## 推薦系統 整理

### 1. 基於內容 (Content-Based)

### <font color="red">2. 協同過濾 (Collaborative)</font>

    1. 基於【用戶】的協同過濾 (User-based)

    2. 基於【物品】的協同過濾 (Item-based)
    
### 3. 混和推薦 (Hybrid)

---

這篇內容會著重在協同過濾

---

In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
# data 
critics = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
                         'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
                         'The Night Listener': 3.0},
           'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
                            'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 3.5},
           'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
                                'Superman Returns': 3.5, 'The Night Listener': 4.0},
           'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                            'The Night Listener': 4.5, 'Superman Returns': 4.0,
                            'You, Me and Dupree': 2.5},
           'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                            'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 2.0},
           'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                             'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
           'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}}

data = pd.DataFrame(critics) # for User-based
dataItem = data.T            # for Item-based

### 定義如何計算相似程度

1. Euclidean Distance 歐式距離

2. Pearson Similarity 相關係數

In [3]:
## return the similarity score from two person

def sim_distance(data,person1,person2):
    """
    1. Euclidean Distance 歐式距離
    --
    
    using Euclidean Distance to find score
    """
    square = (data[person1] - data[person2])**2 # (x1-y1)^2 , (x2-y2)^2 , ..., (xn-yn)^2
    sumOfsquare =  np.sum(square.fillna(0))     # 把上述相加
    distance = np.sqrt(sumOfsquare)             # 開根號
    score = 1/(1 + distance)
    return score


def sim_pearson(data,p1,p2):
    """
    2. Pearson Similarity 相關係數
    --
    
    return pearson similarity between two persons
    """
    p1Lists = data[p1] # Series
    p2Lists = data[p2] # Series
    # choose not null index in each personal Lists
    p1index = p1Lists[p1Lists.notnull()].index
    p2index = p2Lists[p2Lists.notnull()].index
    # pcikup common index 
    commonIndex = pd.Index([e for e in p1index if e in p2index]) 
    return np.corrcoef(p1Lists[commonIndex],p2Lists[commonIndex])[0,1]


def top_match(data, person, n=5, similarity=sim_pearson):
    """
    similarity 要用 Euclidean Distance 還是 Pearson Correlation
    --
    
    return the best match for pearson from the data, numbers of result (n) is optional
    """
    scores = [(other,similarity(data,other,person)) for other in data if other!=person] 
    scores.sort(key=lambda x : x, reverse=True)
    return scores[:n]

## 1. 基於【用戶】的協同過濾 (User-based) - 以 Toby 為例

> ## 用別人的偏好程度(評分紀錄)，去推測我的偏好程度

Assume 某一個用戶其中有幾部電影看過了，其中幾部還沒看過

<font color="red">【想要從其他用戶看過的電影去推測，試圖找出該用戶在還沒看過的電影當中最推薦的 (User-Based)】</font>


1. 準備【用戶評分】資料 : 用戶對電影的評分df (s.t. User在column)

2. 準備相似程度的衡量(User-Based) : 【用戶之間相關係數】，只保留正的值在上面

3. 計算每個未看電影的推薦程度 :

評分方面 : 著眼用戶評分資料，著重在其他用戶對該部還沒看過的電影評分
相關係數佔比方面 : 著眼其他人對Toby正的相關係數

ex. 我還沒看過玩命關頭
> $\Sigma$ [ (其他用戶對玩命關頭的評分)  dot  (在玩命關頭的這個row, 第i個用戶的相關係數佔比)  ]


ex. Toby 還沒看過 Just My Luck, Lady in the Water, The Night Listener

Just My Luck 的推薦程度 = (3 $*$ 0.99/3.18) + (3 $*$ 0.38/3.18) + ( 0 ) + (4.5 $*$ 0.89/3.18) + (3 $*$ 0.92/3.18) + (3 $*$ 0.66/3.18) = 2.53


Lady in the Water 的推薦程度 = (2.5 $*$ 0.99/2.95) + (3 $*$ 0.38/2.95) + ( 0 ) + ( 0 ) + (3 $*$ 0.92/2.95) + (3 $*$ 0.66/2.95) = 2.83


The Night Listener 的推薦程度 = (3 $*$ 0.99/3.84) + (3 $*$ 0.38/3.84) + ( 0 ) + (4.5 $*$ 0.92/3.84) + (3 $*$ 0.92/3.84) + (3 $*$ 0.66/3.84) = 3.35



In [4]:
user = "Toby"
data

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick LaSalle,Jack Matthews,Toby
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN,NaN
Lady in the Water,2.5,3.0,2.5,NaN,3.0,3.0,NaN
Snakes on a Plane,3.5,3.5,3.0,3.5,4.0,4.0,4.5
Superman Returns,3.5,5.0,3.5,4.0,3.0,5.0,4.0
The Night Listener,3.0,3.0,4.0,4.5,3.0,3.0,NaN
"You, Me and Dupree",2.5,3.5,NaN,2.5,2.0,3.5,1.0


In [5]:
def getSimilarityOfUsers(data, person, similarity=sim_pearson):
    # similarity to person
    sim = {person: {other:similarity(data,other,person) for other in data if other!=person}}
    # 每個人之間的相似度(相關係數)
    sim_person = pd.DataFrame(sim)
    sim_person_include = sim_person[sim_person[person]>0].index #剛好人名就都是index
    sim_person = sim_person[sim_person[person]>0]               #取出和這個人正相關的那一群人(還有他們跟他之間的相關係數)
    return sim_person
getSimilarityOfUsers(data, user, similarity=sim_pearson)

,Toby
Claudia Puig,0.893405
Gene Seymour,0.381246
Jack Matthews,0.662849
Lisa Rose,0.991241
Mick LaSalle,0.924473


這邊有一群人，

有

一

堆

電

影



大家都對有看過的電影去評分，但總是會有那種沒看過的電影就無法評分(別擔心 就能靠推薦系統，從剩下你沒看過的電影裡面，挑出最讚的來推薦你)

個別看每個人總有沒看過所以沒評分的 nan 嘛，

1. 我需要先計算每個人的相似程度(衡量方式: 歐式距離 或 相關係數) : 
> 計算相關係數的時候，要用兩兩用戶都一起看過的電影評分來算喔，其中一人沒看過就略過那部電影，不考慮

## <font color="red">推薦系統實作(二)-以用戶為基礎</font>

2. 從你沒看過的電影裡面 推薦你系統覺得最適合你的一部電影

        首先列出所有我沒看過的電影
        
        ex.
        我沒看過玩命關頭
        
        (1) 所以，先找出跟我相似性(相關係數)為正的人，從那群人留下"有看過玩命關頭"的人和評分
        
        (2) 再來，計算每一個跟你正相關又有評玩命關頭的人，
            ([他跟你的相關係數(當權重吧)] dot [他對玩命關頭的評分]) / (留下所有的人 跟你的相關係數總和)
        
        (3) 最後，迴圈跑完每個你沒看過的電影推薦分數之後，由高到低排，最高的就代表是最推薦你的電影囉!
        

### 即文中提到的 相似性權重

> 對Toby來說，電影中有The Night Listener, Just My Luck,Lady in the Water還沒有評分。利用每個使用者對該影片評分與Toby的相似性作加乘，以Night為例



                  相似性   The Night Listener   相似性*Night


        Rose       0.99       3.0                  2.97 
        Seymour    0.38       3.0                  1.14 
        Puig       0.89       4.5                  4.02 
        LaSalle    0.92       3                    2.77 
        Mathews    0.66       3                    1.99 
        總分                                       12.89 
        相似度加總  3.84   
        總分/相似度加總                             3.35 (3.34765625啦)

能如此推測對Toby來說The Nigh Listener得分為3.35 (3.34765625啦)

阿其他電影也用這樣的計算過程得到一個推薦(相似程度)的分數，最高的那部電影就推薦給你囉!


In [6]:
# The Nigh Listener
(0.99/3.84 * 3) + (0.38/3.84 * 3) + (      0      ) + (0.89/3.84 * 4.5) + (0.92/3.84 * 3) + (0.66/3.84 * 3)

3.34765625

In [7]:
# Lady in the Water
(0.99/2.95 * 2.5) + (0.38/2.95 * 3) + (      0      ) + (      0      ) + (0.92/2.95 * 3) + (0.66/2.95 * 3)

2.8322033898305086

In [8]:
# The Nigh Listener
(0.99/3.18 * 3) + (0.38/3.18 * 1.5) + (      0      ) + (0.89/3.18 * 3) + (0.92/3.18 * 2) + (      0      )

2.531446540880503

In [9]:
def getRecommendations(data, person, similarity=sim_pearson):
    
    """
    # Gets recommendations for a person by using a weighted average of every other user's rankings
    """


    # similarity to person
    sim = {person: {other:similarity(data,other,person) for other in data if other!=person}
          }


    # ignore scores lower than 0
    sim_person = pd.DataFrame(sim)
    sim_person_include = sim_person[sim_person[person]>0].index #剛好人名就都是index
    sim_person =         sim_person[sim_person[person]>0]       #取出和這個人正相關的那一群人(還有他們跟他之間的相關係數)


    # item needed to be recommened to the person
    dataperson = data[person]                                   #該用戶對每部電影評分的那條series
    itemRecommendation = dataperson[dataperson.isnull()].index  #列出該用戶沒有評價過的電影 (沒看過才需要被推薦嘛!)

    
    rankings = []
    for item in itemRecommendation: 
        scores_from_other = data.ix[item]                        #ex.我沒看過玩命關頭   #迴圈跑每一個該用戶沒有評價過的電影 (沒看過才需要被推薦嘛!)
        scores_from_other = scores_from_other[sim_person_include]#ex.找跟我相關係數為正的那一些人，他們看玩命關頭的評分為何  #用跟該用戶相關係數為正的那群人 來對該用戶沒看過的電影計算推薦程度  exclude ignoring similarity lower than 0

        scores_from_other = scores_from_other[np.isfinite(scores_from_other)]#ex.從那群人裡只留下"有對玩命關頭有評分"的人及其評分  #score from other but not NaN in person
                         # sim_person就是你跟所有人之間的相關係數的dict
        sim_person_count = sim_person.ix[scores_from_other.index]#ex.從留下的人找他們跟我之間的相關係數(當作推薦的權重)

        #ex.第一個跟你正相關又有評玩命關頭的人，([他跟你的相關係數(當權重吧)] dot [他對玩命關頭的評分]) / (留下所有的人 跟你的相關係數總和)
        normalized_star = np.dot(scores_from_other,sim_person_count)/np.sum(sim_person_count) # 算完其實是個Series: 你 跟你被推薦的評分
        normalized_star = normalized_star[0] # 從那series只取出推薦分數
        rankings.append( (item,normalized_star))
        rankings.sort(reverse=True) # 迴圈跑完每個你沒看過的電影推薦分數之後，由高到低排，最高的就代表是最推薦你的電影囉!

    return rankings

In [10]:
getRecommendations(data, user, similarity = sim_pearson)

[('The Night Listener', 3.3477895267131013),
 ('Lady in the Water', 2.8325499182641614),
 ('Just My Luck', 2.530980703765564)]

## 2. 基於【物品】的協同過濾 (Item-based) - 以 Toby 為例

> ## 用自己歷史的偏好程度(評分紀錄)，去推測對尚未接觸過的商品的偏好程度

Assume 某一個用戶其中有幾部電影看過了，其中幾部還沒看過

<font color="red">【想要從該用戶看過的電影去推測，試圖找出在還沒看過的電影當中最推薦的 (Item-Based)】</font>


1. 準備【用戶評分】資料 : 用戶對電影的評分df.T (s.t. 電影在column)

2. 準備相似程度的衡量(Item-Based) : 【電影之間相關係數】的df，只保留正的值在上面

3. 計算每個未看電影的推薦程度 :

評分方面 : 著眼用戶評分資料，著重在特定用戶(Toby)那個row

相關係數佔比方面 : 著眼特定電影那個row

ex. 我還沒看過玩命關頭
> $\Sigma$ [ (其他第i部看過的電影的評分)  dot  (第i部看過的電影對玩命關頭的相關係數佔比)  ]


ex. Toby 還沒看過 Lady in the Water


Lady in the Water 的推薦程度 = 0 + 0 + (4.5 $*$ 0.76/1.57) + (4 $*$ 0.48/1.57) + 0 + (1 $*$ 0.33/1.57) = 3.61



In [11]:
user = "Toby"
dataItem # data.T

,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,"You, Me and Dupree"
Lisa Rose,3.0,2.5,3.5,3.5,3.0,2.5
Gene Seymour,1.5,3.0,3.5,5.0,3.0,3.5
Michael Phillips,NaN,2.5,3.0,3.5,4.0,NaN
Claudia Puig,3.0,NaN,3.5,4.0,4.5,2.5
Mick LaSalle,2.0,3.0,4.0,3.0,3.0,2.0
Jack Matthews,NaN,3.0,4.0,5.0,3.0,3.5
Toby,NaN,NaN,4.5,4.0,NaN,1.0


In [12]:
def calculateSimiliarItems(data,item,n=10,similarity=sim_pearson):

     # calculate the similiarity between item and other items : 這是Superman Returns和其他電影的相似程度(相關係數)

     #[('You, Me and Dupree', 0.65795169495976902),
     # ('Lady in the Water', 0.48795003647426655),
     # ('Snakes on a Plane', 0.11180339887498947),
     # ('The Night Listener', -0.1798471947990542),
     # ('Just My Luck', -0.42289003161103106)]

    """
    # 記得前面是用similarity to person
    # 那時候是算出 其他人對於跟用戶的相似程度(相關係數)
    sim = {person: {other: similarity(data,other,person) for other in data if other!=person}
          }
    """           # ↑↑↑ 其實只要把上面紅字那邊的person 改成item之後，這段key-value的value部分就是底下的result  
    result = {other: similarity(data,other,item) for other in data if other!=item}
    # result.sort(reverse=True,key=lambda (_,x):x)
    

    return result

# 列出所有電影之間的相似程度(相關係數)的 df ， 自己跟自己的相關係數是 nan ，其餘在 [-1, 1]
itemsSimilarity = {item:calculateSimiliarItems(dataItem,item) for item in dataItem}
df_sim_items = pd.DataFrame(itemsSimilarity)
df_sim_items

,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,"You, Me and Dupree"
Just My Luck,NaN,-0.944911,-0.333333,-0.422890,0.555556,-0.485662
Lady in the Water,-0.944911,NaN,0.763763,0.487950,-0.612372,0.333333
Snakes on a Plane,-0.333333,0.763763,NaN,0.111803,-0.566352,-0.645497
Superman Returns,-0.422890,0.487950,0.111803,NaN,-0.179847,0.657952
The Night Listener,0.555556,-0.612372,-0.566352,-0.179847,NaN,-0.250000
"You, Me and Dupree",-0.485662,0.333333,-0.645497,0.657952,-0.250000,NaN


## <font color="red">推薦系統實作(三)-以商品為基礎</font>

延續之前實作(二)的問題，如果使用以用戶為基礎的相似度，來作判斷會出現如簡介所說

表單上大部分的用戶欄位是空的

必須用戶同時上線，才能計算用戶的相似性


## 以商品為基礎

對<font color="red">Toby</font>來說的商品推薦，利用皮爾森相似度計算每個商品之間的相似性。然後針對不同評分給予商品分數，
                    
                                              (cloumn 放該用戶【沒有】看過的3部電影)
                    
    (row 放該用戶【有】看過的3部電影)

        電影	            評分   Night相似度   評分*Night相似度 Lady相似度  評分*Lady相似度  Luck相似度  評分*Luck相似度
    Snakes on a Plane      4.5    -0.56            -              0.76          3.42          -              -
    Superman Return        4      -0.18            -              0.48          1.92          -              -
    You, Me and Dupree     1      -0.25            -              0.33          0.33          -              -
    
    總分                                                                        5.67
    相似度加總                                                     1.57
    
    總分/相似度之和                                                      (5.67/1.57 = 3.61)

不推薦商品相似度 <0 的品項。所以我們只會推薦Lady in the Water(與之前的結果比較很不一樣)

In [13]:
# Lady in the Water
0 + 0 + (4.5 * 0.76/1.57) + (4 * 0.48/1.57) + 0 + (1 * 0.33/1.57)

3.611464968152866

## <font color="red">基於物品協同過濾(Item-based)的推薦算法</font>

可參考網址：https://kknews.cc/tech/yn29pzn.html

    優缺點:
    Item-based算法原理是給用戶推薦那些和他之前喜歡的物品類似的物品，在推薦方面更加個性化，反映了用戶自己的興趣傳承。
    在用戶有新行為時，一定會導致推薦結果的實時變化。

    適用範圍：長尾物品豐富，用戶個性化需求強烈的領域。


    - 優點：
    可以利用用戶的歷史行為給用戶做推薦解釋，可以令用戶比較信服。比較適用在物品較少、物品更新速度不會很快、用戶的興趣比較固定和持久的情況下。

    - 缺點：
    無法避免用戶冷啟動，需要等待用戶接觸物品後才給他推薦相關物品，沒有辦法在不離線更新物品相似度表的情況下將新物品推薦給用戶。


> 因為現在主角要是物品不是人(用戶)了，所以把上述的data Transpose，把物品(電影)放到 column 那邊


1. 依樣先定義相似程度的公式 (歐式距離 或 相關係數)


2. df_sim_items : 列出所有電影之間的相似程度(相關係數)的 df ， 自己跟自己的相關係數是 nan ，其餘在 [-1, 1]

<font color="dodgerblue">tips : 用 defaultdict  (rankings = defaultdict(float))</font>

    3. df_sim_items 紀錄每部電影之間的相似程度(相關係數)
       df_sim_items[recommendItem][userscore]查該用戶【沒有看過】的那部電影，和【有看過】的某部電影 之間的相似程度(相關係數)
       
4. 一樣，只保留相似程度(相關係數)為正的那些，
   用剩下那群物品的相關係數佔比當作是權重 dot 有看那部電影的評分， 去計算那部沒看過的電影的推薦分數


5. 選出最高推薦分數者

In [14]:
"""
### Item based Recommendation ####
"""
def calculateSimiliarItems(data,item,n=10,similarity=sim_pearson):

     # calculate the similiarity between item and other items : 這是Superman Returns和其他電影的相似程度(相關係數)

     #[('You, Me and Dupree', 0.65795169495976902),
     # ('Lady in the Water', 0.48795003647426655),
     # ('Snakes on a Plane', 0.11180339887498947),
     # ('The Night Listener', -0.1798471947990542),
     # ('Just My Luck', -0.42289003161103106)]

    """
    # 記得前面是用similarity to person
    # 那時候是算出 其他人對於跟用戶的相似程度(相關係數)
    sim = {person: {other: similarity(data,other,person) for other in data if other!=person}
          }
    """           # ↑↑↑ 其實只要把上面紅字那邊的person 改成item之後，這段key-value的value部分就是底下的result  
    result = {other: similarity(data,other,item) for other in data if other!=item}
    # result.sort(reverse=True,key=lambda (_,x):x)
    

    return result

# 列出所有電影之間的相似程度(相關係數)的 df ， 自己跟自己的相關係數是 nan ，其餘在 [-1, 1]
itemsSimilarity = {item:calculateSimiliarItems(dataItem,item) for item in dataItem}
df_sim_items = pd.DataFrame(itemsSimilarity)

def getRecommendedItem(dataItem,user):

    # input:
    scoreByUser = dataItem.ix[user]                  # locate該用戶對每個電影評分的那條row
    scoreByUser = scoreByUser[scoreByUser.notnull()] # pickup not null element for user

    scoreByUserIndex = scoreByUser.index             # 列出該用戶【有】評過分的電影名字


    ## not in user's comment 
    itemsRecommendForUser = dataItem.ix[user]
    itemsRecommendForUser = itemsRecommendForUser[itemsRecommendForUser.isnull()] # 跟上面那段只差在.isnull()
    itemsRecommendForUserIndex = itemsRecommendForUser.index # 列出該用戶【沒有】評過分的電影名字

    # 當普通的一般字典沒有那個key但你偏要選他的話，會報錯 ; 
    # defaultdict 就是，預設像str()=='', int()==0，選不出來時就放預設值去檔，就不會報錯囉!      
    rankings = defaultdict(float)
    for recommendItem in itemsRecommendForUserIndex: # itemsRecommendForUserIndex 列出該用戶【沒有】評過分的電影名字
        sim_sum =0
        for userscore in scoreByUserIndex:           # scoreByUserIndex           列出該用戶【有】評過分的電影名字
            #df_sim_items 紀錄每部電影之間的相似程度(相關係數)
            #df_sim_items[recommendItem][userscore]查該用戶【沒有看過】的那部電影，和【有看過】的某部電影 之間的相似程度(相關係數)
            if df_sim_items[recommendItem][userscore] > 0: 
                sim = df_sim_items[recommendItem][userscore]
                rankings[recommendItem] += sim*scoreByUser[userscore] # 分數 dot 相關係數所佔比例的權重(搭配底下的除以sim_sum)
                sim_sum += sim
        if sim_sum!=0:
            rankings[recommendItem] = rankings[recommendItem]/sim_sum

    return rankings

In [15]:
getRecommendedItem(dataItem, user)

defaultdict(float, {'Lady in the Water': 3.6100310668021787})

## 推薦系統實作(一)- 線性迴歸

In [16]:
#! encoding: utf8 


import numpy as np
import pandas as pd

## datasets #####

datasets2 = dataset={
         'Lisa Rose': {'Lady in the Water': 2.5,
                       'Snakes on a Plane': 3.5,
                       'Just My Luck': 3.0,
                       'Superman Returns': 3.5,
                       'You, Me and Dupree': 2.5,
                       'The Night Listener': 3.0},
         'Gene Seymour': {'Lady in the Water': 3.0,
                          'Snakes on a Plane': 3.5,
                          'Just My Luck': 1.5,
                          'Superman Returns': 5.0,
                          'The Night Listener': 3.0,
                          'You, Me and Dupree': 3.5},
 
        'Michael Phillips': {'Lady in the Water': 2.5,
                             'Snakes on a Plane': 3.0,
                             'Superman Returns': 3.5,
                             'The Night Listener': 4.0},
        'Claudia Puig': {'Snakes on a Plane': 3.5,
                         'Just My Luck': 3.0,
                         'The Night Listener': 4.5,
                         'Superman Returns': 4.0,
                         'You, Me and Dupree': 2.5},
        'Mick LaSalle': {'Lady in the Water': 3.0,
                         'Snakes on a Plane': 4.0,
                         'Just My Luck': 2.0,
                         'Superman Returns': 3.0,
                         'The Night Listener': 3.0,
                         'You, Me and Dupree': 2.0},
       'Jack Matthews': {'Lady in the Water': 3.0,
                         'Snakes on a Plane': 4.0,
                         'The Night Listener': 3.0,
                         'Superman Returns': 5.0,
                         'You, Me and Dupree': 3.5},
      'Toby': {'Snakes on a Plane':4.5,
               'You, Me and Dupree':1.0,
               'Superman Returns':4.0}}

datasets1 = {'Alice':{'Love at last':5,
                     'Romance forever':5,
                     'Nonstop car chase':0,
                     'Swords vs Karate':0},
             'Bob':{'Love at last':5,
                     'Cute puppies of love':4,
                     'Nonstop car chase':0,
                     'Swords vs Karate':0},
             'Carol':{'Love at last':0,
                     'Cute puppies of love':0,
                     'Nonstop car chase':5,
                     'Swords vs Karate':5},
             'Dave':{'Love at last':0,
                     'Romance forever':0,
                     'Nonstop car chase':4,
                     },
             'Eva':{

             }
            }

### 


data = pd.DataFrame(datasets1)

totalUsersNumber = data.shape[1]  ## numbers of users

totalMoviesNumber = data.shape[0] ##


featuresNumber = 2 # features used:

                    # 2 is good for datasets1, 5 is good for datasets2.


## Gradient Descent with mean normalization and Low rank factorization

## algorithm detail: 

## see here (http://www.holehouse.org/mlclass/16_Recommender_Systems.html)



X = abs(np.random.randn(featuresNumber,totalMoviesNumber)) # initalize X movie matrix with random

Ymean = data.mean(axis=1)# mean score for each movie

Ytilde = data.subtract(Ymean,axis=0)
# Xtilt = X.subtract(Xmean,axis=0) 


Theta = np.random.randn(featuresNumber,totalUsersNumber)
DeltaY = (np.dot(X.T,Theta) - Ytilde).fillna(0)

alpha = 10**-1 # learning rate

regul = 0.1 # regularization factor

err = 1
iterNo =0

while iterNo<1000 and err>10**-4:
    '''
    Using Gradient descent method to solve Ymean
    '''

    Xiter = X - alpha*(np.dot(Theta,DeltaY.T) + regul*X)
    ThetaIter = Theta - alpha*(np.dot(X,DeltaY) + regul*Theta)
    Ypred = np.dot(Xiter.T,ThetaIter) 
    DeltaY = (Ypred - Ytilde).fillna(0) ## IMPORTANT: 保持沒有資料的地方誤差為零

    err = max(np.max((DeltaY**2).cumsum()))

    X = Xiter
    Theta = ThetaIter
    iterNo +=1
    
    print('iterNo:{}, error:{}'.format(iterNo,err))

dataPredict = pd.DataFrame(Ypred,index=data.index,columns=data.columns)
dataPredict = dataPredict.add(Ymean,axis=0)
print('\n===========================================')
print('predict star label is: \n')
print('{}'.format(data))
print('\n===========================================')
print('predict star label is: \n')
print('{}'.format(dataPredict))
print('===========================================')

iterNo:1, error:26.723732017761208
iterNo:2, error:19.2347748494605
iterNo:3, error:9.11664277464741
iterNo:4, error:3.464531035708398
iterNo:5, error:1.7682876970904013
iterNo:6, error:1.078971418876166
iterNo:7, error:0.7925982439895103
iterNo:8, error:0.6483252781300846
iterNo:9, error:0.5739381086349609
iterNo:10, error:0.5275216440387818
iterNo:11, error:0.4976493163567452
iterNo:12, error:0.4756900428034269
iterNo:13, error:0.4594539047907248
iterNo:14, error:0.44642948181535835
iterNo:15, error:0.4360458835718603
iterNo:16, error:0.4298099713294627
iterNo:17, error:0.4350738399664664
iterNo:18, error:0.4399760164628707
iterNo:19, error:0.4446883457816675
iterNo:20, error:0.4490612229154624
iterNo:21, error:0.4531763292570731
iterNo:22, error:0.45699031376248545
iterNo:23, error:0.46054912175735824
iterNo:24, error:0.46384965131991984
iterNo:25, error:0.46692263419820146
iterNo:26, error:0.46977901752719486
iterNo:27, error:0.47244194815357776
iterNo:28, error:0.47492605429001733

iterNo:262, error:0.016354274497375768
iterNo:263, error:0.01635140206569964
iterNo:264, error:0.016348545234291294
iterNo:265, error:0.01634570495503471
iterNo:266, error:0.016342882120013558
iterNo:267, error:0.01634007756408642
iterNo:268, error:0.01633729206736349
iterNo:269, error:0.01633452635758713
iterNo:270, error:0.016331781112421596
iterNo:271, error:0.016329056961654223
iterNo:272, error:0.016326354489309684
iterNo:273, error:0.016323674235683715
iterNo:274, error:0.01632101669929596
iterNo:275, error:0.016318382338767916
iterNo:276, error:0.01631577157462521
iterNo:277, error:0.01631318479103141
iterNo:278, error:0.016310622337450932
iterNo:279, error:0.016308084530247586
iterNo:280, error:0.016305571654219954
iterNo:281, error:0.01630308396407407
iterNo:282, error:0.016300621685839966
iterNo:283, error:0.01629818501822852
iterNo:284, error:0.016295774133936625
iterNo:285, error:0.016293389180898357
iterNo:286, error:0.016291030283485712
iterNo:287, error:0.016288697543662

iterNo:519, error:0.016138533631858153
iterNo:520, error:0.01613848736860835
iterNo:521, error:0.016138441950857087
iterNo:522, error:0.016138397363299594
iterNo:523, error:0.01613835359090361
iterNo:524, error:0.016138310618905288
iterNo:525, error:0.01613826843280326
iterNo:526, error:0.016138227018355638
iterNo:527, error:0.016138186361573683
iterNo:528, error:0.016138146448719307
iterNo:529, error:0.016138107266299304
iterNo:530, error:0.016138068801061604
iterNo:531, error:0.01613803103999127
iterNo:532, error:0.016137993970305985
iterNo:533, error:0.016137957579452314
iterNo:534, error:0.016137921855101246
iterNo:535, error:0.016137886785145217
iterNo:536, error:0.016137852357692974
iterNo:537, error:0.01613781856106683
iterNo:538, error:0.016137785383798158
iterNo:539, error:0.01613775281462449
iterNo:540, error:0.016137720842485374
iterNo:541, error:0.01613768945651858
iterNo:542, error:0.016137658646057824
iterNo:543, error:0.016137628400627605
iterNo:544, error:0.016137598709

iterNo:786, error:0.016136034020270508
iterNo:787, error:0.0161360337012136
iterNo:788, error:0.016136033388071694
iterNo:789, error:0.016136033080735527
iterNo:790, error:0.01613603277909715
iterNo:791, error:0.01613603248305113
iterNo:792, error:0.016136032192493786
iterNo:793, error:0.016136031907323482
iterNo:794, error:0.016136031627440407
iterNo:795, error:0.016136031352746192
iterNo:796, error:0.01613603108314531
iterNo:797, error:0.016136030818542618
iterNo:798, error:0.016136030558846066
iterNo:799, error:0.01613603030396466
iterNo:800, error:0.016136030053808963
iterNo:801, error:0.01613602980829118
iterNo:802, error:0.01613602956732585
iterNo:803, error:0.016136029330828312
iterNo:804, error:0.01613602909871572
iterNo:805, error:0.016136028870907048
iterNo:806, error:0.0161360286473222
iterNo:807, error:0.01613602842788327
iterNo:808, error:0.016136028212512997
iterNo:809, error:0.01613602800113622
iterNo:810, error:0.01613602779367868
iterNo:811, error:0.01613602759006808
i

## 推薦系統實作(二)-以用戶為基礎

在實作一中，推薦系統的作法是，想像使用者與電影名稱構成一張大表單。假定電影風格()與個人評分權重()存在線性關係()。盡可能去fit表單中已知的數據，然後藉由最小化誤差函數得到最佳的近似解()。

但我們也可以利用計算每個使用者的相似程度，作進一步相似性的推薦。

In [17]:
critics = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
                         'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
                         'The Night Listener': 3.0},
           'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
                            'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 3.5},
           'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
                                'Superman Returns': 3.5, 'The Night Listener': 4.0},
           'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                            'The Night Listener': 4.5, 'Superman Returns': 4.0,
                            'You, Me and Dupree': 2.5},
           'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                            'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 2.0},
           'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                             'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
           'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}}


data = pd.DataFrame(critics)
data

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick LaSalle,Jack Matthews,Toby
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN,NaN
Lady in the Water,2.5,3.0,2.5,NaN,3.0,3.0,NaN
Snakes on a Plane,3.5,3.5,3.0,3.5,4.0,4.0,4.5
Superman Returns,3.5,5.0,3.5,4.0,3.0,5.0,4.0
The Night Listener,3.0,3.0,4.0,4.5,3.0,3.0,NaN
"You, Me and Dupree",2.5,3.5,NaN,2.5,2.0,3.5,1.0


這邊有一群人，

有

一

堆

電

影



大家都對有看過的電影去評分，但總是會有那種沒看過的電影就無法評分(別擔心 就能靠推薦系統，從剩下你沒看過的電影裡面，挑出最讚的來推薦你)

個別看每個人總有沒看過所以沒評分的 nan 嘛，

1. 我需要先計算每個人的相似程度(衡量方式: 歐式距離 或 相關係數) : 
> 計算相關係數的時候，要用兩兩用戶都一起看過的電影評分來算喔，其中一人沒看過就略過那部電影，不考慮

## <font color="red">推薦系統實作(二)-以用戶為基礎</font>

2. 從你沒看過的電影裡面 推薦你系統覺得最適合你的一部電影

        首先列出所有我沒看過的電影
        
        ex.
        我沒看過玩命關頭
        
        (1) 所以，先找出跟我相似性(相關係數)為正的人，從那群人留下"有看過玩命關頭"的人和評分
        
        (2) 再來，計算每一個跟你正相關又有評玩命關頭的人，
            ([他跟你的相關係數(當權重吧)] dot [他對玩命關頭的評分]) / (留下所有的人 跟你的相關係數總和)
        
        (3) 最後，迴圈跑完每個你沒看過的電影推薦分數之後，由高到低排，最高的就代表是最推薦你的電影囉!
        

### 即文中提到的 相似性權重

> 對Toby來說，電影中有The Night Listener, Just My Luck,Lady in the Water還沒有評分。利用每個使用者對該影片評分與Toby的相似性作加乘，以Night為例



                  相似性   The Night Listener   相似性*Night


        Rose       0.99       3.0                  2.97 
        Seymour    0.38       3.0                  1.14 
        Puig       0.89       4.5                  4.02 
        LaSalle    0.92       3                    2.77 
        Mathews    0.66       3                    1.99 
        總分                                       12.89 
        相似度加總  3.84   
        總分/相似度加總                             3.35 (3.34765625啦)

能如此推測對Toby來說The Nigh Listener得分為3.35 (3.34765625啦)

阿其他電影也用這樣的計算過程得到一個推薦(相似程度)的分數，最高的那部電影就推薦給你囉!


In [18]:
(0.99/3.84 * 3) + (0.38/3.84 * 3) + (0.89/3.84 * 4.5) + (0.92/3.84 * 3) + (0.66/3.84 * 3)

3.34765625

<font color = "red">如何定義兩個用戶的相似程度，有常見的兩種方法</font>

# 1. 歐幾里得距離(Euclidean Distance)


In [19]:
data = pd.DataFrame(datasets2)

# pearson_recommendation.py
def sim_distance(data,person1,person2):
    
    """
    # using Euclidean distance to find score
    """

    square = (data[person1] - data[person2])**2 # (x1-y1)^2 , (x2-y2)^2 , ..., (xn-yn)^2
    sumOfsquare =  np.sum(square.fillna(0))     # 把上述相加
    distance = np.sqrt(sumOfsquare)             # 開根號
    score = 1/(1 + distance)

    return score

In [20]:
data["Toby"]

Just My Luck          NaN
Lady in the Water     NaN
Snakes on a Plane     4.5
Superman Returns      4.0
The Night Listener    NaN
You, Me and Dupree    1.0
Name: Toby, dtype: float64

In [21]:
data.head()

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick LaSalle,Jack Matthews,Toby
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN,NaN
Lady in the Water,2.5,3.0,2.5,NaN,3.0,3.0,NaN
Snakes on a Plane,3.5,3.5,3.0,3.5,4.0,4.0,4.5
Superman Returns,3.5,5.0,3.5,4.0,3.0,5.0,4.0
The Night Listener,3.0,3.0,4.0,4.5,3.0,3.0,NaN


In [22]:
# 如果要計算 Toby 與 Rose 的相似性
sim_distance(data, "Toby", "Lisa Rose")

0.3483314773547883

## 2. 皮爾森相關性(Pearson correlation)
> 只計算相對距離存在一個問題，如果Toby和Rose對影片的口味是一致的。但是Rose相對比較保守，會比Toby給的評分都低1~2分。這時候使用相關性會得到比較好的結果。

在numpy裡可以直接使用np.corrcoef(list1,list2)就能計算covariance矩陣。

In [23]:
# pearson_recommendation
def sim_pearson(data,p1,p2):

    """
    # return pearson similarity between two persons
    """
    
    p1Lists = data[p1] # Series
    p2Lists = data[p2] # Series


    # choose not null index in each personal Lists
    p1index = p1Lists[p1Lists.notnull()].index
    p2index = p2Lists[p2Lists.notnull()].index

    # pcikup common index 
    commonIndex = pd.Index([e for e in p1index if e in p2index]) 
    
    
    return np.corrcoef(p1Lists[commonIndex],p2Lists[commonIndex])[0,1]

In [24]:
# 如果要計算 Toby 與 Rose 的相似性
sim_pearson(data, "Toby", "Lisa Rose")

0.9912407071619301

## 相似性權重
對Toby來說，電影中有The Night Listener, Just My Luck,Lady in the Water還沒有評分。利用每個使用者對該影片評分與Toby的相似性作加乘，以Night為例

http://ihong-blog.logdown.com/posts/481703-recommended-system-b

In [25]:
## using pearson distance to evalute the similarity

## collaborative filtering based on 1) users 2)items 


# 2016.02.05


# module 

from collections import defaultdict
import pandas as pd
import numpy as np

### data ####

critics = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
                         'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
                         'The Night Listener': 3.0},
           'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
                            'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 3.5},
           'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
                                'Superman Returns': 3.5, 'The Night Listener': 4.0},
           'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                            'The Night Listener': 4.5, 'Superman Returns': 4.0,
                            'You, Me and Dupree': 2.5},
           'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                            'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 2.0},
           'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                             'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
           'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}}


data = pd.DataFrame(critics)
dataItem = data.T
## return the similarity score from two person


# pearson_recommendation.py
def sim_distance(data,person1,person2):
    
    """
    # using Euclidean Distance to find score
    """

    square = (data[person1] - data[person2])**2 # (x1-y1)^2 , (x2-y2)^2 , ..., (xn-yn)^2
    sumOfsquare =  np.sum(square.fillna(0))     # 把上述相加
    distance = np.sqrt(sumOfsquare)             # 開根號
    score = 1/(1 + distance)

    return score





# pearson_recommendation
def sim_pearson(data,p1,p2):
    
    """
    # return pearson similarity between two persons
    """
    
    p1Lists = data[p1] # Series
    p2Lists = data[p2] # Series

    # choose not null index in each personal Lists
    p1index = p1Lists[p1Lists.notnull()].index
    p2index = p2Lists[p2Lists.notnull()].index

    # pcikup common index 
    commonIndex = pd.Index([e for e in p1index if e in p2index]) 
    
    return np.corrcoef(p1Lists[commonIndex],p2Lists[commonIndex])[0,1]





def top_match(data, person, n=5, similarity=sim_pearson): # similarity要用 Euclidean Distance 還是 Pearson Correlation

    """
    # return the best match for pearson from the data, 
    # numbers of result(n) is optional
    """

    scores = [(other,similarity(data,other,person)) for other in data if other!=person] 
    scores.sort(key=lambda x : x, reverse=True)

    return scores[:n]





def getRecommendations(data, person, similarity=sim_pearson):
    
    """
    # Gets recommendations for a person by using a weighted average of every other user's rankings
    """


    # similarity to person
    sim = {person: {other:similarity(data,other,person) for other in data if other!=person}
          }


    # ignore scores lower than 0
    sim_person = pd.DataFrame(sim)
    sim_person_include = sim_person[sim_person[person]>0].index #剛好人名就都是index
    sim_person = sim_person[sim_person[person]>0]               #取出和這個人正相關的那一群人(還有他們跟他之間的相關係數)


    # item needed to be recommened to the person
    dataperson = data[person]                                   #該用戶對每部電影評分的那條series
    itemRecommendation = dataperson[dataperson.isnull()].index  #列出該用戶沒有評價過的電影 (沒看過才需要被推薦嘛!)

    
    rankings = []
    for item in itemRecommendation: 
        scores_from_other = data.ix[item]                        #ex.我沒看過玩命關頭   #迴圈跑每一個該用戶沒有評價過的電影 (沒看過才需要被推薦嘛!)
        scores_from_other = scores_from_other[sim_person_include]#ex.找跟我相關係數為正的那一些人，他們看玩命關頭的評分為何  #用跟該用戶相關係數為正的那群人 來對該用戶沒看過的電影計算推薦程度  exclude ignoring similarity lower than 0

        scores_from_other = scores_from_other[np.isfinite(scores_from_other)]#ex.從那群人裡只留下"有對玩命關頭有評分"的人及其評分  #score from other but not NaN in person
                         # sim_person就是你跟所有人之間的相關係數的dict
        sim_person_count = sim_person.ix[scores_from_other.index]#ex.從留下的人找他們跟我之間的相關係數(當作推薦的權重)

        #ex.第一個跟你正相關又有評玩命關頭的人，([他跟你的相關係數(當權重吧)] dot [他對玩命關頭的評分]) / (留下所有的人 跟你的相關係數總和)
        normalized_star = np.dot(scores_from_other,sim_person_count)/np.sum(sim_person_count) # 算完其實是個Series: 你 跟你被推薦的評分
        normalized_star = normalized_star[0] # 從那series只取出推薦分數
        rankings.append( (item,normalized_star))
        rankings.sort(reverse=True) # 迴圈跑完每個你沒看過的電影推薦分數之後，由高到低排，最高的就代表是最推薦你的電影囉!

    return rankings

In [26]:
data.head()

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick LaSalle,Jack Matthews,Toby
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN,NaN
Lady in the Water,2.5,3.0,2.5,NaN,3.0,3.0,NaN
Snakes on a Plane,3.5,3.5,3.0,3.5,4.0,4.0,4.5
Superman Returns,3.5,5.0,3.5,4.0,3.0,5.0,4.0
The Night Listener,3.0,3.0,4.0,4.5,3.0,3.0,NaN


In [27]:
getRecommendations(data, "Toby", similarity = sim_pearson)

[('The Night Listener', 3.3477895267131013),
 ('Lady in the Water', 2.8325499182641614),
 ('Just My Luck', 2.530980703765564)]

In [28]:
### data ####

critics = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
                         'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
                         'The Night Listener': 3.0},
           'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
                            'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 3.5},
           'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
                                'Superman Returns': 3.5, 'The Night Listener': 4.0},
           'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                            'The Night Listener': 4.5, 'Superman Returns': 4.0,
                            'You, Me and Dupree': 2.5},
           'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                            'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 2.0},
           'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                             'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
           'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}}


data = pd.DataFrame(critics)
dataItem = data.T

dataItem

,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,"You, Me and Dupree"
Lisa Rose,3.0,2.5,3.5,3.5,3.0,2.5
Gene Seymour,1.5,3.0,3.5,5.0,3.0,3.5
Michael Phillips,NaN,2.5,3.0,3.5,4.0,NaN
Claudia Puig,3.0,NaN,3.5,4.0,4.5,2.5
Mick LaSalle,2.0,3.0,4.0,3.0,3.0,2.0
Jack Matthews,NaN,3.0,4.0,5.0,3.0,3.5
Toby,NaN,NaN,4.5,4.0,NaN,1.0


## <font color="red">推薦系統實作(三)-以商品為基礎</font>

延續之前實作(二)的問題，如果使用以用戶為基礎的相似度，來作判斷會出現如簡介所說

表單上大部分的用戶欄位是空的

必須用戶同時上線，才能計算用戶的相似性


## 以商品為基礎

對<font color="red">Toby</font>來說的商品推薦，利用皮爾森相似度計算每個商品之間的相似性。然後針對不同評分給予商品分數，
                    
                                              (cloumn 放該用戶【沒有】看過的3部電影)
                    
    (row 放該用戶【有】看過的3部電影)

        電影	            評分   Night相似度   評分*Night相似度 Lady相似度  評分*Lady相似度  Luck相似度  評分*Luck相似度
    Snakes on a Plane      4.5    -0.56            -              0.76          3.42          -              -
    Superman Return        4      -0.18            -              0.48          1.92          -              -
    You, Me and Dupree     1      -0.25            -              0.33          0.33          -              -
    
    總分                                                                        5.67
    相似度加總                                                     1.57
    
    總分/相似度之和                                                      (5.67/1.57 = 3.61)

不推薦商品相似度 <0 的品項。所以我們只會推薦Lady in the Water(與之前的結果比較很不一樣)

## <font color="red">基於物品協同過濾(Item-based)的推薦算法</font>

可參考網址：https://kknews.cc/tech/yn29pzn.html

    優缺點:
    Item-based算法原理是給用戶推薦那些和他之前喜歡的物品類似的物品，在推薦方面更加個性化，反映了用戶自己的興趣傳承。
    在用戶有新行為時，一定會導致推薦結果的實時變化。

    適用範圍：長尾物品豐富，用戶個性化需求強烈的領域。


    - 優點：
    可以利用用戶的歷史行為給用戶做推薦解釋，可以令用戶比較信服。比較適用在物品較少、物品更新速度不會很快、用戶的興趣比較固定和持久的情況下。

    - 缺點：
    無法避免用戶冷啟動，需要等待用戶接觸物品後才給他推薦相關物品，沒有辦法在不離線更新物品相似度表的情況下將新物品推薦給用戶。


> 因為現在主角要是物品不是人(用戶)了，所以把上述的data Transpose，把物品(電影)放到 column 那邊


1. 依樣先定義相似程度的公式 (歐式距離 或 相關係數)


2. df_sim_items : 列出所有電影之間的相似程度(相關係數)的 df ， 自己跟自己的相關係數是 nan ，其餘在 [-1, 1]

<font color="dodgerblue">tips : 用 defaultdict  (rankings = defaultdict(float))</font>

    3. df_sim_items 紀錄每部電影之間的相似程度(相關係數)
       df_sim_items[recommendItem][userscore]查該用戶【沒有看過】的那部電影，和【有看過】的某部電影 之間的相似程度(相關係數)
       
4. 一樣，只保留相似程度(相關係數)為正的那些，
   用剩下那群物品的相關係數佔比當作是權重 dot 有看那部電影的評分， 去計算那部沒看過的電影的推薦分數


5. 選出最高推薦分數者

In [29]:
# pearson_recommendation.py
#-----

## using pearson distance to evalute the similarity

## collaborative filtering based on 1) users 2)items 


# 2016.02.05


# module 

from collections import defaultdict
import pandas as pd
import numpy as np

### data ####

critics = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
                         'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
                         'The Night Listener': 3.0},
           'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
                            'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 3.5},
           'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
                                'Superman Returns': 3.5, 'The Night Listener': 4.0},
           'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                            'The Night Listener': 4.5, 'Superman Returns': 4.0,
                            'You, Me and Dupree': 2.5},
           'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                            'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                            'You, Me and Dupree': 2.0},
           'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                             'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
           'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}}


data = pd.DataFrame(critics)
dataItem = data.T


## return the similarity score from two person
def sim_distance(data,person1,person2):
    
    """
    # using Euclidean distance to find score
    """
    square = (data[person1] - data[person2])**2 # (x1-y1)^2 , (x2-y2)^2 , ..., (xn-yn)^2
    sumOfsquare =  np.sum(square.fillna(0))     # 把上述相加
    distance = np.sqrt(sumOfsquare)             # 開根號
    score = 1/(1 + distance)

    return score


def sim_pearson(data,p1,p2):

    """
    # return pearson similarity between two persons
    """
    p1Lists = data[p1] # Series
    p2Lists = data[p2] # Series

    # choose not null index in each personal Lists
    p1index = p1Lists[p1Lists.notnull()].index
    p2index = p2Lists[p2Lists.notnull()].index

    # pcikup common index 
    commonIndex = pd.Index([e for e in p1index if e in p2index]) 
    
    return np.corrcoef(p1Lists[commonIndex],p2Lists[commonIndex])[0,1]






"""
### Item based Recommendation ####
"""

def calculateSimiliarItems(data,item,n=10,similarity=sim_pearson):

     # calculate the similiarity between item and other items : 這是Superman Returns和其他電影的相似程度(相關係數)

     #[('You, Me and Dupree', 0.65795169495976902),
     # ('Lady in the Water', 0.48795003647426655),
     # ('Snakes on a Plane', 0.11180339887498947),
     # ('The Night Listener', -0.1798471947990542),
     # ('Just My Luck', -0.42289003161103106)]

    """
    # 記得前面是用similarity to person
    # 那時候是算出 其他人對於跟用戶的相似程度(相關係數)
    sim = {person: {other: similarity(data,other,person) for other in data if other!=person}
          }
    """           # ↑↑↑ 其實只要把上面紅字那邊的person 改成item之後，這段key-value的value部分就是底下的result  
    result = {other: similarity(data,other,item) for other in data if other!=item}
    # result.sort(reverse=True,key=lambda (_,x):x)
    

    return result

# 列出所有電影之間的相似程度(相關係數)的 df ， 自己跟自己的相關係數是 nan ，其餘在 [-1, 1]
itemsSimilarity = {item:calculateSimiliarItems(dataItem,item) for item in dataItem}
df_sim_items = pd.DataFrame(itemsSimilarity)

def getRecommendedItem(dataItem,user):

    # input:
    scoreByUser = dataItem.ix[user]                  # locate該用戶對每個電影評分的那條row
    scoreByUser = scoreByUser[scoreByUser.notnull()] # pickup not null element for user

    scoreByUserIndex = scoreByUser.index             # 列出該用戶【有】評過分的電影名字


    ## not in user's comment 
    itemsRecommendForUser = dataItem.ix[user]
    itemsRecommendForUser = itemsRecommendForUser[itemsRecommendForUser.isnull()] # 跟上面那段只差在.isnull()
    itemsRecommendForUserIndex = itemsRecommendForUser.index # 列出該用戶【沒有】評過分的電影名字

    # 當普通的一般字典沒有那個key但你偏要選他的話，會報錯 ; 
    # defaultdict 就是，預設像str()=='', int()==0，選不出來時就放預設值去檔，就不會報錯囉!      
    rankings = defaultdict(float)
    for recommendItem in itemsRecommendForUserIndex: # itemsRecommendForUserIndex 列出該用戶【沒有】評過分的電影名字
        sim_sum =0
        for userscore in scoreByUserIndex:           # scoreByUserIndex           列出該用戶【有】評過分的電影名字
            #df_sim_items 紀錄每部電影之間的相似程度(相關係數)
            #df_sim_items[recommendItem][userscore]查該用戶【沒有看過】的那部電影，和【有看過】的某部電影 之間的相似程度(相關係數)
            if df_sim_items[recommendItem][userscore] > 0: 
                sim = df_sim_items[recommendItem][userscore]
                rankings[recommendItem] += sim*scoreByUser[userscore] # 分數 dot 相關係數所佔比例的權重(搭配底下的除以sim_sum)
                sim_sum += sim
        if sim_sum!=0:
            rankings[recommendItem] = rankings[recommendItem]/sim_sum

    return rankings

In [30]:
getRecommendedItem(dataItem,"Toby")

defaultdict(float, {'Lady in the Water': 3.6100310668021787})

In [31]:
data

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick LaSalle,Jack Matthews,Toby
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN,NaN
Lady in the Water,2.5,3.0,2.5,NaN,3.0,3.0,NaN
Snakes on a Plane,3.5,3.5,3.0,3.5,4.0,4.0,4.5
Superman Returns,3.5,5.0,3.5,4.0,3.0,5.0,4.0
The Night Listener,3.0,3.0,4.0,4.5,3.0,3.0,NaN
"You, Me and Dupree",2.5,3.5,NaN,2.5,2.0,3.5,1.0


In [32]:
for p2 in data.T :
    print("電影: ", p2)
    print("Superman Returns跟那部電影的相似性(相關係數): ", sim_pearson(data.T,"Superman Returns", p2))

電影:  Just My Luck
Superman Returns跟那部電影的相似性(相關係數):  -0.42289003161103106
電影:  Lady in the Water
Superman Returns跟那部電影的相似性(相關係數):  0.48795003647426655
電影:  Snakes on a Plane
Superman Returns跟那部電影的相似性(相關係數):  0.11180339887498947
電影:  Superman Returns
Superman Returns跟那部電影的相似性(相關係數):  1.0
電影:  The Night Listener
Superman Returns跟那部電影的相似性(相關係數):  -0.1798471947990542
電影:  You, Me and Dupree
Superman Returns跟那部電影的相似性(相關係數):  0.6579516949597689


In [33]:
dataItem.ix["Toby"]

Just My Luck          NaN
Lady in the Water     NaN
Snakes on a Plane     4.5
Superman Returns      4.0
The Night Listener    NaN
You, Me and Dupree    1.0
Name: Toby, dtype: float64

In [34]:
dataItem.columns

Index(['Just My Luck', 'Lady in the Water', 'Snakes on a Plane',
       'Superman Returns', 'The Night Listener', 'You, Me and Dupree'],
      dtype='object')

In [35]:
for item in dataItem :
    print(item)

Just My Luck
Lady in the Water
Snakes on a Plane
Superman Returns
The Night Listener
You, Me and Dupree


In [36]:
def calculateSimiliarItems(data,item,n=10,similarity=sim_pearson):


     # calculate the similiarity between item and other items : 這是Superman Returns和其他電影的相似程度(相關係數)
     #[('You, Me and Dupree', 0.65795169495976902),
     # ('Lady in the Water', 0.48795003647426655),
     # ('Snakes on a Plane', 0.11180339887498947),
     # ('The Night Listener', -0.1798471947990542),
     # ('Just My Luck', -0.42289003161103106)]



    result = {other: similarity(data,other,item) for other in data if other!= item
             }
    # result.sort(reverse=True,key=lambda (_,x):x)

    return result

itemsSimilarity = {item:calculateSimiliarItems(dataItem,item) for item in dataItem}
# print(itemsSimilarity)
df_sim_items = pd.DataFrame(itemsSimilarity)

df_sim_items

,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,"You, Me and Dupree"
Just My Luck,NaN,-0.944911,-0.333333,-0.422890,0.555556,-0.485662
Lady in the Water,-0.944911,NaN,0.763763,0.487950,-0.612372,0.333333
Snakes on a Plane,-0.333333,0.763763,NaN,0.111803,-0.566352,-0.645497
Superman Returns,-0.422890,0.487950,0.111803,NaN,-0.179847,0.657952
The Night Listener,0.555556,-0.612372,-0.566352,-0.179847,NaN,-0.250000
"You, Me and Dupree",-0.485662,0.333333,-0.645497,0.657952,-0.250000,NaN


In [37]:
df_sim_items["Superman Returns"]["Lady in the Water"]

0.48795003647426655